In [1]:
import nltk
import json

from nltk.stem import WordNetLemmatizer

import pickle
import random
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

from keras.models import load_model

lemmatizer = WordNetLemmatizer()
ignore = ['!','?']

In [2]:
file = open("C:/francais.json").read()
data = json.loads(file)

words_Repository=[]
corpus=[]
classes=[]

lemmatizer = WordNetLemmatizer()
ignore = ['!','?']

for record in data['info']:
    for request in record['requests']:
        word= nltk.word_tokenize(request)
        word= [lemmatizer.lemmatize(w.lower()) for w in word if w not in ignore]
        words_Repository.extend(word)
        t = record['title']
        corpus.append((word,t))
        if t not in classes:
            classes.append(t)

In [3]:
words= list(set(words_Repository))

words=sorted(words)

classes=sorted(classes)

pickle.dump(words,open('C:/chatbot/frwords.pkl','wb'))
pickle.dump(classes,open('C:/chatbot/frclasses.pkl','wb'))
pickle.dump(corpus,open('C:/chatbot/frcorpus.pkl','wb'))

In [4]:
words= pickle.load(open('C:/chatbot/frwords.pkl','rb'))
classes= pickle.load(open('C:/chatbot/frclasses.pkl','rb'))
corpus= pickle.load(open('C:/chatbot/frcorpus.pkl','rb'))
model = load_model('C:/chatbot/frmodel.h5')

# Digital Representation:

In [6]:
dataSet=[]

for request,class_r in corpus:
    input_vec = [0]*len(words)
    output_vec = [0]*len(classes)
    
    for w in request:
        input_vec[words.index(w)]=1
    
    output_vec[classes.index(class_r)]=1
    
    dataSet.append([input_vec,output_vec])

random.shuffle(dataSet)
dataSet= np.array(dataSet,dtype=object)

x=list(dataSet[:,0])
y=list(dataSet[:,1])

# Build Model

In [7]:
def createModel(input_size,output_size):
    model = Sequential()
    model.add(Dense(128,input_shape=(input_size,),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_size,activation='softmax'))
    
    return model

model = createModel(len(words),len(classes))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               17152     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 13)                845       
                                                                 
Total params: 26,253
Trainable params: 26,253
Non-trainable params: 0
_________________________________________________________________


In [8]:
### train the model

sgd = SGD(learning_rate=0.01,momentum=0.8,nesterov=True)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(np.array(x),np.array(y), epochs=200, batch_size=3,verbose=1)

Epoch 1/200
17/17 [==============================] - 1s 5ms/step - loss: 2.5649 - accuracy: 0.0588
Epoch 2/200
17/17 [==============================] - 0s 3ms/step - loss: 2.5490 - accuracy: 0.0784
Epoch 3/200
17/17 [==============================] - 0s 3ms/step - loss: 2.4252 - accuracy: 0.1569
Epoch 4/200
17/17 [==============================] - 0s 3ms/step - loss: 2.4065 - accuracy: 0.1961
Epoch 5/200
17/17 [==============================] - 0s 3ms/step - loss: 2.2827 - accuracy: 0.2941
Epoch 6/200
17/17 [==============================] - 0s 3ms/step - loss: 2.2079 - accuracy: 0.3333
Epoch 7/200
17/17 [==============================] - 0s 3ms/step - loss: 2.2272 - accuracy: 0.3529
Epoch 8/200
17/17 [==============================] - 0s 3ms/step - loss: 2.1065 - accuracy: 0.3529
Epoch 9/200
17/17 [==============================] - 0s 3ms/step - loss: 2.0165 - accuracy: 0.3725
Epoch 10/200
17/17 [==============================] - 0s 3ms/step - loss: 1.8722 - accuracy: 0.4314
Epoch 11/

17/17 [==============================] - 0s 3ms/step - loss: 0.0989 - accuracy: 0.9804
Epoch 84/200
17/17 [==============================] - 0s 3ms/step - loss: 0.1752 - accuracy: 0.9412
Epoch 85/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1581 - accuracy: 0.9804
Epoch 86/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2402 - accuracy: 0.9020
Epoch 87/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1913 - accuracy: 0.9412
Epoch 88/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1040 - accuracy: 0.9608
Epoch 89/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1523 - accuracy: 0.9608
Epoch 90/200
17/17 [==============================] - 0s 3ms/step - loss: 0.0826 - accuracy: 0.9804
Epoch 91/200
17/17 [==============================] - 0s 3ms/step - loss: 0.0949 - accuracy: 1.0000
Epoch 92/200
17/17 [==============================] - 0s 3ms/step - loss: 0.2113 - accuracy: 0.9412
Epoch 93/200


In [9]:
model.save('C:/chatbot/frmodel.h5')

In [10]:
### preprocessing methods

##sen = "hello, how are you?"

def senWords(sen):
    swords=nltk.word_tokenize(sen)
    swords=[lemmatizer.lemmatize(w.lower())for w in swords if w not in ignore]
    return swords

def digitize(sen,words):
    input_vec = [0] * len(words)
    swords = senWords(sen)
    
    for sw in swords:
        if sw in words:
            indx = words.index(sw)
            input_vec[indx] = 1
    return input_vec

def predictClass(sen,words,model):
    sample = digitize(sen,words)
    value = model.predict(np.array([sample]))[0]

    indx = list(value).index(max(value))
    return classes[indx]

def getResponse(sen,words,classes,model):
    tag = predictClass(sen,words,model)
    for record in data['info']:
        if record['title'] == tag :
            return random.choice(record['responses'])

# GUI Python Tkinter

In [16]:
from tkinter import *

gui = Tk()

## configuration:
gui.title('ChatBot version -1-')
gui.geometry('400x500')
gui.resizable(width=False,height=False)

##Components:
chatArea = Text(gui,font='Arial')
chatArea.config(state=DISABLED)

scrollBar = Scrollbar(gui,command = chatArea.yview)

def chatting():
    sen=entryBox.get('1.0',END).strip()
    entryBox.delete('1.0',END)
    
    if sen !='':
        ##if sen in qQuestions:
            chatArea.config(state=NORMAL)
            chatArea.insert(END,"You: "+sen+ "\n\n")
        
            res=getResponse(sen,words,classes,model)
        
            chatArea.insert(END,"Bot :"+res+ "\n\n")
        
            chatArea.config(state=DISABLED)
            chatArea.yview(END)
        

f=('Arial',18,'bold')
chatButton = Button(gui,bd=0,command=chatting,width=7,text='send',fg='white',font=f,bg='darkorange',activebackground='gray')

entryBox = Text(gui,font='Arial')

##location
chatArea.place(x=6,y=6,height=380,width=380)
scrollBar.place(x=380,y=6,height=380,width=20)
chatButton.place(x=6,y=400,height=90)
entryBox.place(x=110,y=400,height=90,width=280)
gui.mainloop()


1/1 [==============================] - 0s 53ms/step
